In [29]:
import os
import pickle
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
import feature_eng_function as f_eng

from datetime import datetime
from matplotlib import pyplot as plt
from PCA_function import pca_data100
from IPython.core.display import display, HTML
from sklearn import ensemble
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import GridSearchCV

%matplotlib inline
display(HTML("<style>.container { width:100% !important; }</style>"))

## Import the training and testing data

In [2]:
forest_train = pd.read_csv("data/train.csv")
forest_test = pd.read_csv("data/test.csv")
forest_base_train = pd.read_csv("data/train_eng.csv")
forest_base_test = pd.read_csv("data/test_eng.csv")
forest_100_train = pd.read_csv("data/train_100.csv")
forest_100_test = pd.read_csv("data/test_100.csv")

## Create arrays for each of the data sets

In [3]:
y_train = forest_train['Cover_Type']
ID = forest_test['Id']

X_train = forest_train[[col for col in forest_train.columns.tolist() if col not in ['Id','Cover_Type']]].values
X_train_base = forest_base_train[[col for col in forest_base_train.columns.tolist() if col not in ['Id','Cover_Type']]].values
X_train_100 = forest_100_train[[col for col in forest_100_train.columns.tolist() if col not in ['Id','Cover_Type']]].values

X_test = forest_test[[col for col in forest_test.columns.tolist() if col not in ['Id','Cover_Type']]].values
X_test_base = forest_base_test[[col for col in forest_base_test.columns.tolist() if col not in ['Id','Cover_Type']]].values
X_test_100 = forest_100_test[[col for col in forest_100_test.columns.tolist() if col not in ['Id','Cover_Type']]].values


## Import the pickles

In [5]:
# Gradient Boosting Models
optimized_100_cv5_GBM = pickle.load( open('pickles/optimized_100_cv5_GBM.p', 'rb') )
optimized_100_cv10_GBM = pickle.load( open('pickles/optimized_100_cv10_GBM.p', 'rb') )
optimized_100_default_GBM = pickle.load( open('pickles/optimized_100_default_GBM.p', 'rb') )
optimized_base_cv5_GBM = pickle.load( open('pickles/optimized_base_cv5_GBM.p', 'rb') )
optimized_base_cv10_GBM  = pickle.load( open('pickles/optimized_base_cv10_GBM.p', 'rb') )
optimized_base_default_GBM  = pickle.load( open('pickles/optimized_base_default_GBM.p', 'rb') )
optimized_kaggle_default_GBM = pickle.load( open('pickles/optimized_kaggle_default_GBM.p', 'rb') )
optimized_kaggle_cv5_GBM = pickle.load( open('pickles/optimized_kaggle_cv5_GBM.p', 'rb') )
optimized_kaggle_cv10_GBM = pickle.load( open('pickles/optimized_kaggle_cv10_GBM.p', 'rb') )

# Random Forest Models
optimized_100_cv5_RF = pickle.load( open('pickles/optimized_100_cv5_RF.p', 'rb') )
optimized_100_cv10_RF = pickle.load( open('pickles/optimized_100_cv10_RF.p', 'rb') )
optimized_base_cv5_RF = pickle.load( open('pickles/optimized_base_cv5_RF.p', 'rb') )
optimized_base_cv10_RF = pickle.load( open('pickles/optimized_base_cv10_RF.p', 'rb') )
optimized_kaggle_cv5_RF = pickle.load( open('pickles/optimized_kaggle_cv5_RF.p', 'rb') )
optimized_kaggle_cv10_RF = pickle.load( open('pickles/optimized_kaggle_cv10_RF.p', 'rb') )

#Extra Trees Models
optimized_kaggle_cv10_ET = pickle.load( open('pickles/optimized_kaggle_cv10_ET.p', 'rb') )
optimized_base_cv10_ET = pickle.load( open('pickles/optimized_base_cv10_ET.p', 'rb') )
optimized_100_cv10_ET = pickle.load( open('pickles/optimized_100_cv10_ET.p', 'rb') )


## Create lists of models to test on for each data set

In [32]:
models_100 = [
    ('optimized_100_cv5_GBM',optimized_100_cv5_GBM),
    ('optimized_100_cv10_GBM',optimized_100_cv10_GBM),
    ('optimized_100_default_GBM',optimized_100_default_GBM),
    ('optimized_100_cv5_RF',optimized_100_cv5_RF),
    ('optimized_100_cv10_RF',optimized_100_cv10_RF),
    ('optimized_100_cv10_ET',optimized_100_cv10_ET)
]

models_base = [
    ('optimized_base_cv5_GBM',optimized_base_cv5_GBM),
    ('optimized_base_cv10_GBM',optimized_base_cv10_GBM),
    ('optimized_base_default_GBM',optimized_base_default_GBM),
    ('optimized_base_cv5_RF',optimized_base_cv5_RF),
    ('optimized_base_cv10_RF',optimized_base_cv10_RF),
    ('optimized_base_cv10_ET',optimized_base_cv10_ET)
]

models_kaggle = [
    ('optimized_kaggle_cv5_GBM',optimized_kaggle_cv5_GBM),
    ('optimized_kaggle_cv10_GBM',optimized_kaggle_cv10_GBM),
    ('optimized_kaggle_default_GBM',optimized_kaggle_default_GBM),
    ('optimized_kaggle_cv5_RF',optimized_kaggle_cv5_RF),
    ('optimized_kaggle_cv10_RF',optimized_kaggle_cv10_RF),
    ('optimized_kaggle_cv10_ET',optimized_kaggle_cv10_ET) 
]

## Test each of the models and import into data frame to view results

In [35]:
for i in range(len(models_kaggle)):
    try:
        print (models_kaggle[i][0], "best score: ", models_kaggle[i][1].best_score_)
        print (models_kaggle[i][1].best_params_, "scoring by: ", models_kaggle[i][1].scorer_, "\n")
    except:
        pass
    

optimized_kaggle_cv5_GBM best score:  0.778637566138
{'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1} scoring by:  make_scorer(accuracy_score) 

optimized_kaggle_cv10_GBM best score:  0.777777777778
{'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1} scoring by:  make_scorer(accuracy_score) 

optimized_kaggle_cv5_RF best score:  0.654365079365
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 700} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 

optimized_kaggle_cv10_RF best score:  0.666997354497
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 300} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 

optimized_kaggle_cv10_ET best score:  0.785119047619
{'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 1000} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 



In [36]:
for i in range(len(models_base)):
    try:
        print (models_base[i][0], "best score: ", models_base[i][1].best_score_)
        print (models_base[i][1].best_params_, "scoring by: ", models_base[i][1].scorer_, "\n")
    except:
        pass
    

optimized_base_cv5_GBM best score:  0.780952380952
{'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1} scoring by:  make_scorer(accuracy_score) 

optimized_base_cv10_GBM best score:  0.778439153439
{'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1} scoring by:  make_scorer(accuracy_score) 

optimized_base_cv5_RF best score:  0.650925925926
{'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 100} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 

optimized_base_cv10_RF best score:  0.662764550265
{'criterion': 'gini', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 200} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 

optimized_base_cv10_ET best score:  0.778174603175
{'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 700} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 



In [37]:
for i in range(len(models_100)):
    try:
        print (models_100[i][0], "best score: ", models_100[i][1].best_score_)
        print (models_100[i][1].best_params_, "scoring by: ", models_100[i][1].scorer_, "\n")
    except:
        pass
    

optimized_100_cv5_GBM best score:  0.797883597884
{'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1} scoring by:  make_scorer(accuracy_score) 

optimized_100_cv10_GBM best score:  0.797354497354
{'learning_rate': 0.15, 'max_depth': 5, 'min_child_weight': 1} scoring by:  make_scorer(accuracy_score) 

optimized_100_cv5_RF best score:  0.66541005291
{'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 1000} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 

optimized_100_cv10_RF best score:  0.671362433862
{'criterion': 'entropy', 'max_depth': 5, 'max_features': 'sqrt', 'n_estimators': 1000} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 

optimized_100_cv10_ET best score:  0.811706349206
{'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 1000} scoring by:  <function _passthrough_scorer at 0x114bc30d0> 



In [38]:
for i in range(len(models_kaggle)):
    m = models_kaggle[i][0]
    mod = models_kaggle[i][1]
    filename = str('submissions/' + m + '.csv')
    if os.path.isfile(filename):
        pass
    else:
        results = pd.DataFrame(ID)
        results['Cover_Type'] = mod.predict(X_test)
        results['Cover_Type'] = results['Cover_Type'].astype(int)
        results.to_csv(filename, index=False)
    print (m, "predictions complete!")
    
for i in range(len(models_base)):
    m = models_base[i][0]
    mod = models_base[i][1]
    filename = str('submissions/' + m + '.csv')
    if os.path.isfile(filename):
        pass
    else:
        results = pd.DataFrame(ID)
        results['Cover_Type'] = mod.predict(X_test_base)
        results['Cover_Type'] = results['Cover_Type'].astype(int)
        results.to_csv(filename, index=False)
    print (m, "predictions complete!")
    
for i in range(len(models_100)):
    m = models_100[i][0]
    mod = models_100[i][1]
    filename = str('submissions/' + m + '.csv')
    if os.path.isfile(filename):
        pass
    else:
        results = pd.DataFrame(ID)
        results['Cover_Type'] = mod.predict(X_test_100)
        results['Cover_Type'] = results['Cover_Type'].astype(int)
        results.to_csv(filename, index=False)
    print (m, "predictions complete!")
    

optimized_kaggle_cv5_GBM predictions complete!
optimized_kaggle_cv10_GBM predictions complete!
optimized_kaggle_default_GBM predictions complete!
optimized_kaggle_cv5_RF predictions complete!
optimized_kaggle_cv10_RF predictions complete!
optimized_kaggle_cv10_ET predictions complete!
optimized_base_cv5_GBM predictions complete!
optimized_base_cv10_GBM predictions complete!
optimized_base_default_GBM predictions complete!
optimized_base_cv5_RF predictions complete!
optimized_base_cv10_RF predictions complete!
optimized_base_cv10_ET predictions complete!
optimized_100_cv5_GBM predictions complete!
optimized_100_cv10_GBM predictions complete!
optimized_100_default_GBM predictions complete!
optimized_100_cv5_RF predictions complete!
optimized_100_cv10_RF predictions complete!
optimized_100_cv10_ET predictions complete!
